In [14]:
# Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
import time
from sklearn.metrics import classification_report, accuracy_score
import os
import torch.quantization
import torch.nn.utils.prune as prune

In [2]:
# Constants
tEPOCHS = 3
EPOCHS = 10  # Set distillation epochs to 1 for demonstration purposes


# Load the dataset
data = pd.read_excel('/kaggle/input/ubmec-498r/UBMEC.xlsx')
#data = data.sample(frac=1, random_state=42).reset_index(drop=True)
#data = data.tail(1000)

# Split the data
X = data['text'].astype(str).tolist()
y = data['classes']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Device configuration
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Using device: {device}')

# Utility functions
def encode_texts(texts, tokenizer, max_len=512):
    return tokenizer(texts, padding=True, truncation=True, max_length=max_len, return_tensors='pt')

def calculate_accuracy(preds, labels):
    preds_argmax = torch.argmax(preds, dim=1)
    correct = (preds_argmax == labels).float()
    accuracy = correct.sum() / len(correct)
    return accuracy.item()

# Train Teacher Model (BERT)
#teacher_name = 'bert-base-multilingual-cased'
#tokenizer = BertTokenizer.from_pretrained(teacher_name)
#teacher_model = BertForSequenceClassification.from_pretrained(teacher_name, num_labels=6)

# Load model directly
#from transformers import AutoTokenizer, AutoModelForPreTraining

#model = AutoModelForPreTraining.from_pretrained("csebuetnlp/banglabert")

Using device: cuda


In [3]:
# Load Teacher Model (BERT)
teacher_name = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(teacher_name)
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_name, num_labels=6)
teacher_model.to(device)

# Checkpoint path for the teacher model
checkpoint_path = 'best_teacher_model.pth'

""""
# Load best teacher model checkpoint if it exists
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    teacher_model.load_state_dict(checkpoint['model_state_dict'])
    best_val_acc = checkpoint['val_acc']
    print(f"Loaded best teacher model checkpoint with val_acc: {best_val_acc*100:.4f}%")
"""
train_encodings = encode_texts(X_train, tokenizer)
test_encodings = encode_texts(X_test, tokenizer)

y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], y_train)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], y_test)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(teacher_model.parameters(), lr=1e-5)

# Training function for Teacher Model
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    
    for input_ids, attention_mask, labels in iterator:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        loss = criterion(logits, labels)
        acc = calculate_accuracy(logits, labels)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Evaluation function for Teacher Model
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for input_ids, attention_mask, labels in iterator:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            
            loss = criterion(logits, labels)
            acc = calculate_accuracy(logits, labels)

            all_preds.extend(logits.argmax(dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            epoch_loss += loss.item()
            epoch_acc += acc
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator), all_preds, all_labels

# Initialize the best validation accuracy variables
best_val_acc = 0.0
best_val_acc_student = 0.0

# Early stopping parameters
patience = 3
patience_counter = 0

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [4]:
# Training Teacher Model with Checkpointing and Early Stopping
for epoch in range(tEPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(teacher_model, train_loader, optimizer, criterion)
    val_loss, val_acc, all_preds, all_labels = evaluate(teacher_model, test_loader, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
    
    print(f'Epoch: {epoch+1:02}/{tEPOCHS} | Epoch Time: {int(epoch_mins)}m {int(epoch_secs)}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.4f}%')
    print(f'\tVal Loss: {val_loss:.3f} | Val Acc: {val_acc*100:.4f}%')
    
    # Checkpointing for the best teacher model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': teacher_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
        }, checkpoint_path)
        print(f'Saved new best teacher model with val_acc: {val_acc*100:.4f}%')
        patience_counter = 0  # Reset patience counter
    else:
        patience_counter += 1

    # Early stopping
    if patience_counter >= patience:
        print("Early stopping triggered")
        break


Epoch: 01/3 | Epoch Time: 10m 41s
	Train Loss: 1.529 | Train Acc: 38.8021%
	Val Loss: 1.388 | Val Acc: 45.9449%
Saved new best teacher model with val_acc: 45.9449%
Epoch: 02/3 | Epoch Time: 10m 41s
	Train Loss: 1.263 | Train Acc: 51.7578%
	Val Loss: 1.260 | Val Acc: 51.4881%
Saved new best teacher model with val_acc: 51.4881%
Epoch: 03/3 | Epoch Time: 10m 41s
	Train Loss: 1.072 | Train Acc: 59.9702%
	Val Loss: 1.267 | Val Acc: 52.9390%
Saved new best teacher model with val_acc: 52.9390%


In [5]:
# Generate classification report for Teacher Model
class_names = ['joy', 'disgust', 'anger', 'sadness', 'surprise', 'fear']
teacher_model.load_state_dict(torch.load(checkpoint_path)['model_state_dict'])
_, _, all_preds_teacher, all_labels_teacher = evaluate(teacher_model, test_loader, criterion)
report_teacher = classification_report(all_labels_teacher, all_preds_teacher, target_names=class_names, digits=4)
print("Teacher Model Classification Report:\n", report_teacher)
# Generate classification report for Teacher Model

#report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)
#print("Teacher Model Classification Report:\n", report)


/tmp/ipykernel_36/922558547.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model.load_state_dict(torch.load(checkpoint_path)['model_state_dict'])


Teacher Model Classification Report:
               precision    recall  f1-score   support

         joy     0.4527    0.6721    0.5410       491
     disgust     0.3553    0.3245    0.3392       416
       anger     0.6149    0.3309    0.4303       275
     sadness     0.6897    0.7849    0.7342       674
    surprise     0.4741    0.4898    0.4818       541
        fear     0.6952    0.2509    0.3687       291

    accuracy                         0.5294      2688
   macro avg     0.5470    0.4755    0.4825      2688
weighted avg     0.5442    0.5294    0.5163      2688



In [6]:
# Load the best teacher model for distillation
#teacher_model.load_state_dict(torch.load(checkpoint_path)['model_state_dict'])

# Define a custom distillation model
class DistillationModel(nn.Module):
    def __init__(self, student_model, teacher_model, temperature=2, alpha=0.5):
        super(DistillationModel, self).__init__()
        self.student_model = student_model
        self.teacher_model = teacher_model
        self.temperature = temperature
        self.alpha = alpha

    def forward(self, input_ids, attention_mask):
        # Compute student logits
        student_outputs = self.student_model(input_ids, attention_mask=attention_mask)
        student_logits = student_outputs.logits
        
        # Compute teacher logits (no gradients needed)
        with torch.no_grad():
            teacher_outputs = self.teacher_model(input_ids, attention_mask=attention_mask)
            teacher_logits = teacher_outputs.logits
        
        return student_logits, teacher_logits

# Training and evaluation functions for Student Model
def train_student(model, iterator, optimizer, criterion, temperature=2, alpha=0.5):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    
    for input_ids, attention_mask, labels in iterator:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        
        student_logits, teacher_logits = model(input_ids, attention_mask=attention_mask)
        
        student_loss = criterion(student_logits, labels)
        distillation_loss = nn.KLDivLoss()(nn.functional.log_softmax(student_logits / temperature, dim=1),
                                           nn.functional.softmax(teacher_logits / temperature, dim=1)) * (temperature * temperature)
        loss = alpha * student_loss + (1 - alpha) * distillation_loss
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += calculate_accuracy(student_logits, labels)

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate_student(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    all_preds_student = []
    all_labels_student = []
    
    with torch.no_grad():
        for input_ids, attention_mask, labels in iterator:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            
            student_logits, _ = model(input_ids, attention_mask=attention_mask)
            
            loss = criterion(student_logits, labels)
            acc = calculate_accuracy(student_logits, labels)

            all_preds_student.extend(student_logits.argmax(dim=1).cpu().numpy())
            all_labels_student.extend(labels.cpu().numpy())
            
            epoch_loss += loss.item()
            epoch_acc += acc
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator), all_preds_student, all_labels_student

# Load Student Model (DistilBERT with Sequence Classification Head)
student_name = 'h4g3n/distilbert-mini-multilingual-cased'
student_tokenizer = AutoTokenizer.from_pretrained(student_name)
student_model = AutoModelForSequenceClassification.from_pretrained(student_name, num_labels=6)

student_model.to(device)

distillation_model = DistillationModel(student_model, teacher_model)
distillation_model.to(device)

optimizer_student = optim.AdamW(distillation_model.student_model.parameters(), lr=5e-5)

# Checkpoint path for the student model
checkpoint_path_student = 'best_student_model.pth'


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/208M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at h4g3n/distilbert-mini-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Training Student Model with Checkpointing and Early Stopping
for epoch in range(EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train_student(distillation_model, train_loader, optimizer_student, criterion, temperature=2, alpha=0.5)
    val_loss, val_acc, all_preds_student, all_labels_student = evaluate_student(distillation_model, test_loader, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
    
    print(f'Epoch: {epoch+1:02}/{EPOCHS} | Epoch Time: {int(epoch_mins)}m {int(epoch_secs)}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.4f}%')
    print(f'\tVal Loss: {val_loss:.3f} | Val Acc: {val_acc*100:.4f}%')
    
    # Checkpointing for the best student model
    if val_acc > best_val_acc_student:
        best_val_acc_student = val_acc
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': distillation_model.student_model.state_dict(),
            'optimizer_state_dict': optimizer_student.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
        }, checkpoint_path_student)
        print(f'Saved new best student model with val_acc: {val_acc*100:.4f}%')
        patience_counter = 0  # Reset patience counter
    else:
        patience_counter += 1

    # Early stopping
    if patience_counter >= patience:
        print("Early stopping triggered")
        break


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:2994: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Epoch: 01/10 | Epoch Time: 4m 33s
	Train Loss: 0.873 | Train Acc: 32.5335%
	Val Loss: 1.543 | Val Acc: 36.0491%
Saved new best student model with val_acc: 36.0491%
Epoch: 02/10 | Epoch Time: 4m 33s
	Train Loss: 0.789 | Train Acc: 39.7228%
	Val Loss: 1.548 | Val Acc: 39.3601%
Saved new best student model with val_acc: 39.3601%
Epoch: 03/10 | Epoch Time: 4m 33s
	Train Loss: 0.754 | Train Acc: 43.1827%
	Val Loss: 1.481 | Val Acc: 40.6250%
Saved new best student model with val_acc: 40.6250%
Epoch: 04/10 | Epoch Time: 4m 33s
	Train Loss: 0.724 | Train Acc: 45.4334%
	Val Loss: 1.475 | Val Acc: 40.9970%
Saved new best student model with val_acc: 40.9970%
Epoch: 05/10 | Epoch Time: 4m 33s
	Train Loss: 0.693 | Train Acc: 48.7630%
	Val Loss: 1.469 | Val Acc: 41.6295%
Saved new best student model with val_acc: 41.6295%
Epoch: 06/10 | Epoch Time: 4m 33s
	Train Loss: 0.656 | Train Acc: 52.0089%
	Val Loss: 1.484 | Val Acc: 43.0432%
Saved new best student model with val_acc: 43.0432%
Epoch: 07/10 | E

In [8]:
# Load the best teacher and student model checkpoints for evaluation
teacher_model.load_state_dict(torch.load(checkpoint_path)['model_state_dict'])
distillation_model.student_model.load_state_dict(torch.load(checkpoint_path_student)['model_state_dict'])

# Generate evaluation reports for the best models
_, _, all_preds_teacher, all_labels_teacher = evaluate(teacher_model, test_loader, criterion)
_, _, all_preds_student, all_labels_student = evaluate_student(distillation_model, test_loader, criterion)

# Generate classification reports for both models
report_teacher = classification_report(all_labels_teacher, all_preds_teacher, target_names=class_names, digits=4)
report_student = classification_report(all_labels_student, all_preds_student, target_names=class_names, digits=4)

print("Teacher Model Classification Report:\n", report_teacher)
print("Distilled Student Model Classification Report:\n", report_student)

/tmp/ipykernel_36/3788420965.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model.load_state_dict(torch.load(checkpoint_path)['model_state_dict'])
/tmp/ipykerne

Teacher Model Classification Report:
               precision    recall  f1-score   support

         joy     0.4527    0.6721    0.5410       491
     disgust     0.3553    0.3245    0.3392       416
       anger     0.6149    0.3309    0.4303       275
     sadness     0.6897    0.7849    0.7342       674
    surprise     0.4741    0.4898    0.4818       541
        fear     0.6952    0.2509    0.3687       291

    accuracy                         0.5294      2688
   macro avg     0.5470    0.4755    0.4825      2688
weighted avg     0.5442    0.5294    0.5163      2688

Distilled Student Model Classification Report:
               precision    recall  f1-score   support

         joy     0.4130    0.3625    0.3861       491
     disgust     0.2854    0.2812    0.2833       416
       anger     0.4097    0.3382    0.3705       275
     sadness     0.6121    0.6884    0.6480       674
    surprise     0.3620    0.4436    0.3987       541
        fear     0.3367    0.2302    0.2735   

# Calculating size

In [9]:
import os

def get_model_size_and_params(model):
    param_count = sum(p.numel() for p in model.parameters())
    temp_model_path = "temp_model.pth"
    torch.save(model.state_dict(), temp_model_path)
    model_size = os.path.getsize(temp_model_path)
    os.remove(temp_model_path)
    return param_count, model_size

# Calculate size and param count for the distilled student model
#param_count_student, model_size_student = get_model_size_and_params(distillation_model.student_model)
#print(f"Distilled Student Model - Params: {param_count_student}, Size: {model_size_student / 1e6:.2f}MB")

In [10]:
# Save the distilled student model
#student_model_save_path = "distilled_student_model.pth"
#torch.save(distillation_model.student_model.state_dict(), student_model_save_path)

# Verify the saved model size
#model_size_student = os.path.getsize(student_model_save_path)
#print(f"Distilled Student Model Size: {model_size_student / 1e6:.2f} MB")


In [11]:
# Calculate size and param count before distillation for the student model
param_count_before_distillation, model_size_before_distillation = get_model_size_and_params(teacher_model)
print(f"Original Teacher Model - Params: {param_count_before_distillation}, Size: {model_size_before_distillation / 1e6:.2f}MB")

# Calculate size and param count after distillation for the student model
param_count_after_distillation, model_size_after_distillation = get_model_size_and_params(distillation_model.student_model)
print(f"Distilled Student Model - Params: {param_count_after_distillation}, Size: {model_size_after_distillation / 1e6:.2f}MB")


Original Teacher Model - Params: 177858054, Size: 711.51MB
Distilled Student Model - Params: 52164870, Size: 208.68MB


In [16]:
# Pruning Code
import torch.nn.utils.prune as prune
import transformers

def prune_attention(module, amount):
    for name in ['query', 'key', 'value', 'dense']:
        layer = getattr(module, name)
        prune.l1_unstructured(layer, name='weight', amount=amount)
        if layer.bias is not None:
            prune.l1_unstructured(layer, name='bias', amount=amount)

def prune_model(model, pruning_rate=0.3):
    for module in model.modules():
        if isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=pruning_rate)
            if module.bias is not None:
                prune.l1_unstructured(module, name='bias', amount=pruning_rate)
        elif isinstance(module, transformers.models.bert.modeling_bert.BertSelfAttention):
            prune_attention(module, pruning_rate)

In [17]:
# Apply Pruning to the Distilled Student Model
pruning_rate = 0.3  # Adjust pruning rate as needed
prune_model(distillation_model.student_model, pruning_rate)


In [20]:
# Remove pruning re-parametrization for cleaner model saving
def safe_remove_pruning(module, param_name):
    try:
        prune.remove(module, param_name)
    except ValueError as e:
        print(f"Skipping removal of pruning for {param_name} in {module}: {e}")

for module in distillation_model.student_model.modules():
    if isinstance(module, torch.nn.Linear) or isinstance(module, transformers.models.bert.modeling_bert.BertSelfAttention):
        safe_remove_pruning(module, 'weight')
        if module.bias is not None:
            safe_remove_pruning(module, 'bias')

# Delete zeroed parameters to reduce model size
def delete_zero_parameters(model):
    for name, param in list(model.named_parameters()):
        mask = param != 0
        if not mask.any():
            delattr(model, name)
            
def count_nonzero_weights(model):
    nonzero_count = 0
    total_count = 0
    for param in model.parameters():
        nonzero_count += param.nonzero().size(0)
        total_count += param.numel()
    return nonzero_count, total_count

delete_zero_parameters(distillation_model.student_model)




Skipping removal of pruning for weight in Linear(in_features=384, out_features=384, bias=True): Parameter 'weight' of module Linear(in_features=384, out_features=384, bias=True) has to be pruned before pruning can be removed
Skipping removal of pruning for bias in Linear(in_features=384, out_features=384, bias=True): Parameter 'bias' of module Linear(in_features=384, out_features=384, bias=True) has to be pruned before pruning can be removed
Skipping removal of pruning for weight in Linear(in_features=384, out_features=384, bias=True): Parameter 'weight' of module Linear(in_features=384, out_features=384, bias=True) has to be pruned before pruning can be removed
Skipping removal of pruning for bias in Linear(in_features=384, out_features=384, bias=True): Parameter 'bias' of module Linear(in_features=384, out_features=384, bias=True) has to be pruned before pruning can be removed
Skipping removal of pruning for weight in Linear(in_features=384, out_features=384, bias=True): Parameter 'w

In [21]:
# Evaluate the Pruned Student Model
_, _, all_preds_pruned_student, all_labels_pruned_student = evaluate_student(distillation_model, test_loader, criterion)

# Generate classification report for the Pruned Student Model
report_pruned_student = classification_report(all_labels_pruned_student, all_preds_pruned_student, target_names=class_names, digits=4)
print("Pruned Distilled Student Model Classification Report:\n", report_pruned_student)

# Calculate size and param count after pruning for the student model
param_count_after_pruning_student, model_size_after_pruning_student = get_model_size_and_params(distillation_model.student_model)
print(f"Pruned Distilled Student Model - Params: {param_count_after_pruning_student}, Size: {model_size_after_pruning_student / 1e6:.2f}MB")

# Count non-zero parameters
nonzero_count_pruned_student, total_count_pruned_student = count_nonzero_weights(distillation_model.student_model)
print(f"Non-zero parameters after pruning in Distilled Student Model: {nonzero_count_pruned_student}/{total_count_pruned_student} ({nonzero_count_pruned_student/total_count_pruned_student:.2%})")

Pruned Distilled Student Model Classification Report:
               precision    recall  f1-score   support

         joy     0.4209    0.3523    0.3836       491
     disgust     0.2757    0.2837    0.2796       416
       anger     0.4054    0.3273    0.3622       275
     sadness     0.6107    0.6914    0.6486       674
    surprise     0.3634    0.4473    0.4010       541
        fear     0.3333    0.2268    0.2699       291

    accuracy                         0.4297      2688
   macro avg     0.4016    0.3881    0.3908      2688
weighted avg     0.4234    0.4297    0.4230      2688

Pruned Distilled Student Model - Params: 52164870, Size: 208.68MB
Non-zero parameters after pruning in Distilled Student Model: 50346974/52164870 (96.52%)


In [22]:
import torch
import torch.nn as nn

# Apply dynamic quantization to the pruned model
quantized_model = torch.quantization.quantize_dynamic(
    distillation_model.student_model,  # the pruned model
    {nn.Linear},  # layers to quantize
    dtype=torch.qint8  # quantization data type
)

# Function to calculate model size
def calculate_model_size(model):
    torch.save(model.state_dict(), "temp.pth")
    model_size = os.path.getsize("temp.pth")
    os.remove("temp.pth")
    return model_size

# Calculate size and number of parameters after quantization
param_count_after_quantization = sum(p.numel() for p in quantized_model.parameters())
model_size_after_quantization = calculate_model_size(quantized_model)

print(f"Quantized Pruned Model - Params: {param_count_after_quantization}, Size: {model_size_after_quantization / 1e6:.2f}MB")

# Count non-zero parameters after quantization
nonzero_count_quantized, total_count_quantized = count_nonzero_weights(quantized_model)
print(f"Non-zero parameters after quantization in Pruned Model: {nonzero_count_quantized}/{total_count_quantized} ({nonzero_count_quantized/total_count_quantized:.2%})")


Quantized Pruned Model - Params: 46106496, Size: 190.54MB
Non-zero parameters after quantization in Pruned Model: 46106111/46106496 (100.00%)
